# Part 1

In [1]:
#Let's load the libraries and packages we will require
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline
import requests

### Download and Prepare Toronto neighborhoods data from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_postalcodes = pd.read_html(requests.get(url).content)[0] # Retrieves the table on the wikipedia page
df_postalcodes.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
#Examine size of the dataset
df_postalcodes.shape

(287, 3)

In [4]:
# Preparing the data by removing 'Not assigned' values
df_postalcodes = df_postalcodes[df_postalcodes['Borough']!='Not assigned']
df_postalcodes.shape

(210, 3)

In [5]:
df_not_assigned_neigh = df_postalcodes[df_postalcodes['Neighbourhood']=='Not assigned']
df_not_assigned_neigh

,Postcode,Borough,Neighbourhood


##### Postcodes with 'Not assigned' boroughs were removed. The updated dataset does not contain any borough or neighbourhood values which are 'Not assigned'

All neighborhoods associated with a postal code are brought together in one rows with comma separated format

In [6]:
df_postalcodes1 = df_postalcodes.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
df_postalcodes1.reset_index(level=['Postcode','Borough'], inplace=True)


In [7]:
df_postalcodes1.shape

(103, 3)

# Part 2

#### Next we will import the geographical coordinates of each postal code and add them to the dataset

In [8]:
latlong = pd.read_csv('https://cocl.us/Geospatial_data')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
latlong.rename(columns = {'Postal Code':'Postcode'},inplace=True)

In [10]:
# Joining the Toronto postalcodes dataset with latitude and longitude data
df_combined = pd.merge(df_postalcodes1, latlong, on='Postcode', how='inner')
df_combined.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
df_combined.shape

(103, 5)

In [12]:
df_combined.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df_combined.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

In [13]:
### Here we will segment and cluster the Toronto neighborhoods based on location details imported from FourSquare database

In [14]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Let's select a subset of neighborhoods for further analysis - those that have the string 'Toronto' in their names

In [15]:
df_Toronto = df_combined[df_combined['Borough'].str.contains("Toronto")]
df_Toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
df_Toronto.shape # We fill use these 39 neighborhoods for clustering

(39, 5)

#### Let's visualize the selected neighborhoods on the map of Toronto

In [17]:
# Toronto latitude and longitude values
latitude = 43.65
longitude = -79.35
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Next we will establish a connection with Foursquare database with our credentials and import information about venues located near to each neighborhood location coordinates

In [18]:
# The code was removed by Watson Studio for sharing.

In [93]:
LIMIT=100
#Create function to import venues information for each neighborhood: venue name, location and venue category
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [94]:
Toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [95]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2152, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop


In [161]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,92,92,92,92,92,92
"Brockton,Exhibition Place,Parkdale Village",42,42,42,42,42,42
Business Reply Mail Processing Centre 969 Eastern,23,23,23,23,23,23
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",18,18,18,18,18,18
"Cabbagetown,St. James Town",58,58,58,58,58,58
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,19,19,19,19,19,19


In [162]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 251 uniques categories.


#### Analyze Each Neighborhood

In [163]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Train Station,Tram Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
Toronto_onehot.shape

(2152, 251)

In [165]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Tram Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02000,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.01,0.0,0.0
1,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01087,...,0.0,0.0,0.0,0.021739,0.0,0.0,0.0,0.00,0.0,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.047619,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0


In [166]:
Toronto_grouped.shape

(39, 251)

In [167]:
# Let's create a function to return the most common venue categories for each location
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [168]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Café,Sushi Restaurant,Thai Restaurant,Pizza Place,Gym,Theater,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Café,Hotel,Japanese Restaurant,Seafood Restaurant,Lounge,Vegetarian / Vegan Restaurant,Breakfast Spot,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Yoga Studio,Restaurant,Gift Shop,Sandwich Place,Bakery,Breakfast Spot,Burrito Place,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Burrito Place,Grocery Store,Gym / Fitness Center,Spa,Smoke Shop,Skate Park,Sandwich Place,Restaurant,Recording Studio,Pizza Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Coffee Shop,Airport Lounge,Airport Terminal,Plane,Sculpture Garden,Boat or Ferry,Bar,Boutique,Harbor / Marina
5,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Pizza Place,Italian Restaurant,Pub,Bakery,Pharmacy,Gastropub,Café
6,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Italian Restaurant,Ice Cream Shop
7,"Chinatown,Grange Park,Kensington Market",Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Dessert Shop,Dumpling Restaurant,Mexican Restaurant
8,Christie,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Athletics & Sports,Candy Store,Diner,Nightclub,Baby Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Café,Burger Joint,Gastropub,Ramen Restaurant,Sushi Restaurant,Dance Studio


#### Cluster Neighborhoods

In [173]:
# set number of clusters
kclusters = 6

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++",n_clusters=kclusters, n_init = 25).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

array([0, 0, 0, 5, 0, 0, 0, 0, 5, 0, 0, 5, 5, 0, 0, 5, 0, 2, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 5, 0, 5], dtype=int32)

#### After some trial with K-values for clustering, k=6 seems to provide a healthy differentiation in cluster labels.

In [174]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [175]:
Toronto_merged # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Toy / Game Store,Bakery,Indian Restaurant,Ice Cream Shop,Asian Restaurant,Breakfast Spot,Mexican Restaurant,French Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Pub,Café,Italian Restaurant,Park,Ice Cream Shop,Furniture / Home Store,Grocery Store,Burger Joint
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,5,Café,Park,Sandwich Place,Gym,Farmers Market,Sushi Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Bar,Coffee Shop,Italian Restaurant,Diner,American Restaurant,Bakery,Pizza Place,Gastropub,Brewery
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Lake,Swim School,Wings Joint,Dive Bar,Diner,Discount Store,Distribution Center,Doner Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,5,Pizza Place,Hotel,Pharmacy,Convenience Store,Deli / Bodega,Café,Sandwich Place,Bus Line,Brewery,Breakfast Spot
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Café,Coffee Shop,Mexican Restaurant,Shop & Service,Salon / Barbershop,Restaurant,Rental Car Location,Chinese Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,5,Dessert Shop,Italian Restaurant,Café,Pizza Place,Sandwich Place,Coffee Shop,Indian Restaurant,Sushi Restaurant,Gym,Hotel
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1,Park,Playground,Gym,Dive Bar,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0,Coffee Shop,Italian Restaurant,Restaurant,Sushi Restaurant,Grocery Store,Bagel Shop,Pub,Pizza Place,Gym,German Restaurant


In [176]:
# Drop rows having NaN as cluster labels, if any
Toronto_merged.dropna(axis=0,inplace=True)

In [177]:
Toronto_merged.shape

(39, 16)

In [178]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [179]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Pub,Health Food Store,Toy / Game Store,Bakery,Indian Restaurant,Ice Cream Shop,Asian Restaurant,Breakfast Spot,Mexican Restaurant,French Restaurant
41,East Toronto,0,Greek Restaurant,Coffee Shop,Pub,Café,Italian Restaurant,Park,Ice Cream Shop,Furniture / Home Store,Grocery Store,Burger Joint
43,East Toronto,0,Café,Bar,Coffee Shop,Italian Restaurant,Diner,American Restaurant,Bakery,Pizza Place,Gastropub,Brewery
46,Central Toronto,0,Clothing Store,Sporting Goods Shop,Café,Coffee Shop,Mexican Restaurant,Shop & Service,Salon / Barbershop,Restaurant,Rental Car Location,Chinese Restaurant
49,Central Toronto,0,Coffee Shop,Italian Restaurant,Restaurant,Sushi Restaurant,Grocery Store,Bagel Shop,Pub,Pizza Place,Gym,German Restaurant
51,Downtown Toronto,0,Coffee Shop,Restaurant,Park,Pizza Place,Italian Restaurant,Pub,Bakery,Pharmacy,Gastropub,Café
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Café,Burger Joint,Gastropub,Ramen Restaurant,Sushi Restaurant,Dance Studio
53,Downtown Toronto,0,Coffee Shop,Theater,Bakery,Café,Pub,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Shoe Store
54,Downtown Toronto,0,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Bubble Tea Shop,Tea Room,Theater,Bookstore
55,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Breakfast Spot,Seafood Restaurant,Gym,Cosmetics Shop,Italian Restaurant


#### This first cluster is heavily populated by eateries of different kinds and comprises the most neighborhoods. This is not surprising as we selected the neighborhoods with name 'Toronto' in them, not the suburbs. Let's call this cluster 'Bustling with Light Eats'

In [180]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Park,Playground,Gym,Dive Bar,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
50,Downtown Toronto,1,Park,Trail,Playground,Wings Joint,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


#### We can call this cluster 'Parks/ Recreation'

In [181]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,2,Sushi Restaurant,Bus Line,Jewelry Store,Trail,Wings Joint,Diner,Discount Store,Distribution Center,Dive Bar,Dessert Shop


#### Residential

In [182]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Playground,Garden,Italian Restaurant,Kitchen Supply Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Department Store,Dive Bar


#### Green Urban Neighborhood

In [183]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Bus Line,Park,Lake,Swim School,Wings Joint,Dive Bar,Diner,Discount Store,Distribution Center,Doner Restaurant


#### Easy Commute Neighboorhood

In [184]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,5,Café,Park,Sandwich Place,Gym,Farmers Market,Sushi Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse,Fast Food Restaurant
45,Central Toronto,5,Pizza Place,Hotel,Pharmacy,Convenience Store,Deli / Bodega,Café,Sandwich Place,Bus Line,Brewery,Breakfast Spot
47,Central Toronto,5,Dessert Shop,Italian Restaurant,Café,Pizza Place,Sandwich Place,Coffee Shop,Indian Restaurant,Sushi Restaurant,Gym,Hotel
65,Central Toronto,5,Sandwich Place,Café,Pub,Coffee Shop,Park,Middle Eastern Restaurant,Pizza Place,Cosmetics Shop,Modern European Restaurant,Deli / Bodega
75,Downtown Toronto,5,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Athletics & Sports,Candy Store,Diner,Nightclub,Baby Store
76,West Toronto,5,Pharmacy,Park,Bakery,Coffee Shop,Bar,Supermarket,Brazilian Restaurant,Brewery,Café,Pool
87,East Toronto,5,Burrito Place,Grocery Store,Gym / Fitness Center,Spa,Smoke Shop,Skate Park,Sandwich Place,Restaurant,Recording Studio,Pizza Place


#### Commercial and Restaurants

### End of Assignment